<div dir="rtl">
مرحبا
</div>

In [14]:
!sudo apt install poppler-utils -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [15]:
!mkdir -p ../documents
!mkdir -p ../output/raw
!mkdir -p ../output/cleaned

In [25]:
!wget -O ../documents/aws-cloud-adoption-framework.pdf -nc https://d1.awsstatic.com/whitepapers/International/ar/aws-cloud-adoption-framework.pdf --no-check-certificate

--2024-09-04 11:39:32--  https://d1.awsstatic.com/whitepapers/International/ar/aws-cloud-adoption-framework.pdf
Resolving d1.awsstatic.com (d1.awsstatic.com)... 

108.139.80.115, 2600:9000:2543:b600:19:2ae8:f748:f7e1, 2600:9000:2543:4000:19:2ae8:f748:f7e1, ...
Connecting to d1.awsstatic.com (d1.awsstatic.com)|108.139.80.115|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 1336106 (1.3M) [application/pdf]
Saving to: ‘../documents/aws-cloud-adoption-framework.pdf’

../documents/aws-cl 100%[===================>]   1.27M  1.09MB/s    in 1.2s    

2024-09-04 11:39:35 (1.09 MB/s) - ‘../documents/aws-cloud-adoption-framework.pdf’ saved [1336106/1336106]



In [26]:
document = "../documents/aws-cloud-adoption-framework.pdf"

In [27]:
ls -alh {document}

-rw-r--r-- 1 sagemaker-user users 1.3M Mar  9  2022 ../documents/aws-cloud-adoption-framework.pdf


In [28]:
!pdfinfo {document}

Title:           إطار عمل اعتماد السحابة من AWS
Subject:         إطار عمل اعتماد السحابة من AWS
Keywords:        CAF, Adoption
Author:          Amazon Web Services
Creator:         Microsoft Word
CreationDate:    Fri Mar  4 21:17:33 2022 UTC
ModDate:         Fri Mar  4 21:35:12 2022 UTC
Custom Metadata: no
Metadata Stream: yes
Tagged:          yes
UserProperties:  no
Suspects:        no
Form:            none
JavaScript:      no
Pages:           25
Encrypted:       no
Page size:       612 x 792 pts (letter)
Page rot:        0
File size:       1336106 bytes
Optimized:       no
PDF version:     1.7


In [30]:
filename = document.split("/")[-1].split(".")[0]
raw_text_output_path = f"../output/raw/{filename}.txt"
raw_text_output_path

'../output/raw/aws-cloud-adoption-framework.txt'

In [31]:
!pdftotext -layout {document} {raw_text_output_path}

In [40]:
!head -20 {raw_text_output_path}

 ‫إطار عمل اعتماد السحابة من ‪AWS‬‬
‫التحول الرقمي المدعوم بالسحابة‬
                         ‫ّ‬     ‫تسريع عمليات‬



       ‫تم نشره ألول مرة في فبراير‪/‬شباط ‪2015‬‬

     ‫تم التحديث في ‪ 22‬نوفمبر‪ /‬تشرين الثاني ‪2021‬‬
                                                                                               ‫اإلشعارات‬
‫العمالء مسؤولون عن إجراء تقييمهم المستقل للمعلومات الموجودة في هذا المستند‪ .‬هذا المستند‪( :‬أ) ألغراض إعالمية فقط‪،‬‬
 ‫(ب) ويمثل عروض منتجات ‪ AWS‬وممارساتها الحالية التي تخضع للتغيير بدون إشعار‪( ،‬ج) وال يشكِّل أي التزامات أو‬
    ‫ضمانات من جانب ‪ AWS‬والشركات التابعة لها أو مورديها أو مرخصيها‪ .‬يتم توفير منتجات ‪ AWS‬أو خدماتها «كما‬
       ‫هي» دون ضمانات أو إقرارات أو شروط من أي نوع‪ ،‬سواء كانت صريحة أو ضمنية‪ .‬وتخضع مسؤوليات ‪AWS‬‬
  ‫والتزاماتها نحو عمالئها إلى اتفاقيات ‪ ،AWS‬وال يمثل هذا المستند جز ًءا من أي اتفاقية مبرمة بين ‪ AWS‬وعمالئها وال‬
                                                                                                

In [33]:
with open(raw_text_output_path) as txt:
    text_lines = txt.readlines()

In [34]:
text_lines[:10]

[' \u202bإطار عمل اعتماد السحابة من \u202aAWS\u202c\u202c\n',
 '\u202bالتحول الرقمي المدعوم بالسحابة\u202c\n',
 '                         \u202bّ\u202c     \u202bتسريع عمليات\u202c\n',
 '\n',
 '\n',
 '\n',
 '       \u202bتم نشره ألول مرة في فبراير\u202a/\u202cشباط \u202a2015\u202c\u202c\n',
 '\n',
 '     \u202bتم التحديث في \u202a 22\u202cنوفمبر\u202a /\u202cتشرين الثاني \u202a2021\u202c\u202c\n',
 '\x0c                                                                                               \u202bاإلشعارات\u202c\n']

In [35]:
import re


def clean_line(line):
    unicodes_to_replace = ["\u202a", "\u202b", "\u202c"]
    for unicode in unicodes_to_replace:
        line = line.replace(unicode, "")
    return line


pattern = re.compile(r'\s{10,}')
cleaned_text = ""

for line in text_lines:
    cleaned_line = clean_line(line).strip()
    result = pattern.split(cleaned_line)[::-1]

    if len(result) == 1:
        line = result[0]
        if line:
            cleaned_text += line + "\n"
        else:
            # handle empty lines
            cleaned_text += "\n"

    else:
        format_string = " ".join(["{: >50}"] * len(result))
        # handle multiple columns lines
        cleaned_text += format_string.format(*result) + "\n"
        # print(cleaned_text)
        # cleaned_text += "             ".join(result) + "\n"


cleaned_text_output_path = f"../output/cleaned/{filename}.txt"

with open(cleaned_text_output_path, "w") as txt:
    txt.write(cleaned_text)

In [39]:
!head -20 "../output/cleaned/{filename}.txt"

إطار عمل اعتماد السحابة من AWS
التحول الرقمي المدعوم بالسحابة
ّ     تسريع عمليات



تم نشره ألول مرة في فبراير/شباط 2015

تم التحديث في  22نوفمبر /تشرين الثاني 2021
اإلشعارات
العمالء مسؤولون عن إجراء تقييمهم المستقل للمعلومات الموجودة في هذا المستند .هذا المستند( :أ) ألغراض إعالمية فقط،
(ب) ويمثل عروض منتجات  AWSوممارساتها الحالية التي تخضع للتغيير بدون إشعار( ،ج) وال يشكِّل أي التزامات أو
ضمانات من جانب  AWSوالشركات التابعة لها أو مورديها أو مرخصيها .يتم توفير منتجات  AWSأو خدماتها «كما
هي» دون ضمانات أو إقرارات أو شروط من أي نوع ،سواء كانت صريحة أو ضمنية .وتخضع مسؤوليات AWS
والتزاماتها نحو عمالئها إلى اتفاقيات  ،AWSوال يمثل هذا المستند جز ًءا من أي اتفاقية مبرمة بين  AWSوعمالئها وال
ً
تعديال لها. يمثل



